! pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.10.0+cu111.html

In [4]:
%load_ext autoreload
%autoreload 2

In [7]:
import torch
import torch_scatter
import torch_sparse
#import torch_geometric
import tqdm.auto
import os
from operator import itemgetter
# import tensorflow

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import re
import regex as rex
import math

from collections import defaultdict
import matplotlib.cm as cm

import csv
import function_library_gordon_code as funclib

In [8]:
#path = "/content/drive/MyDrive/output7_new_sentiment.csv"
filenm = "./output_attr_stats.csv"
# from google.colab import drive
# drive.mount('/content/drive')
#in my drive I have the csv file and I will be pulling that from my google drive

In [9]:
emails = pd.read_csv(filenm)
#loads the emails into a pandas dataframe

In [98]:
dates = df['date_sent'].values
count_not_string = 0
valid_dates = []

for date in dates:
    try:
        if not rex.match(r'.*20', date):
            print(date)
        else:
            valid_dates.append(date)
    except:
        count_not_string += 1

print("nb non-strings: ", count_not_string)
np.savetxt("valid_dates", sorted(valid_dates), fmt='%s')

# We find 585 records where the Sent field is not a string. 

nb non-strings:  585


In [49]:
emails['Sent'].isnull().sum()

277

In [92]:
df = emails
df[(df['date_sent'] < '2008') | (df['date_sent'] > '2018')].date_sent

Series([], Name: date_sent, dtype: object)

In [99]:
def choose_month_year(df, year, month):
    min_date = f"{year}-{month}-01"
    max_date = f"{year}-{month}-31"
    month_df = df[(df['date_sent'].str.lower() <= max_date) & (df['date_sent'].str.lower() >= min_date)]
    return month_df

dct = {}
years = [('%4d' % year) for year in range(2008, 2018)]
months = [('%02d' % month) for month in range(1,13)]
for year in years:
    for month in months:
        dct[(year, month)] = choose_month_year(emails, year, month)

nb_rows = 0      
for k,v in dct.items():
    nb_rows += v.shape[0]
    print(k, v.shape)
    
print("tot_nb_rows: ", nb_rows)
print("emails.shape: ", emails.shape)
dct['2013','04'].to_csv("2013-04.csv", index=0)

('2008', '01') (0, 28)
('2008', '02') (0, 28)
('2008', '03') (0, 28)
('2008', '04') (0, 28)
('2008', '05') (0, 28)
('2008', '06') (0, 28)
('2008', '07') (0, 28)
('2008', '08') (4, 28)
('2008', '09') (0, 28)
('2008', '10') (0, 28)
('2008', '11') (0, 28)
('2008', '12') (0, 28)
('2009', '01') (0, 28)
('2009', '02') (0, 28)
('2009', '03') (0, 28)
('2009', '04') (0, 28)
('2009', '05') (0, 28)
('2009', '06') (0, 28)
('2009', '07') (0, 28)
('2009', '08') (0, 28)
('2009', '09') (0, 28)
('2009', '10') (0, 28)
('2009', '11') (0, 28)
('2009', '12') (0, 28)
('2010', '01') (0, 28)
('2010', '02') (0, 28)
('2010', '03') (0, 28)
('2010', '04') (3, 28)
('2010', '05') (0, 28)
('2010', '06') (0, 28)
('2010', '07') (0, 28)
('2010', '08') (0, 28)
('2010', '09') (0, 28)
('2010', '10') (0, 28)
('2010', '11') (0, 28)
('2010', '12') (0, 28)
('2011', '01') (0, 28)
('2011', '02') (0, 28)
('2011', '03') (0, 28)
('2011', '04') (0, 28)
('2011', '05') (0, 28)
('2011', '06') (0, 28)
('2011', '07') (0, 28)
('2011', '0

In [96]:
dct[('2013','11')].date_sent

35       2013-11-14
71       2013-11-07
376      2013-11-08
411      2013-11-08
663      2013-11-22
            ...    
34787    2013-11-08
35270    2013-11-07
35361    2013-11-15
35393    2013-11-25
35590    2013-11-25
Name: date_sent, Length: 391, dtype: object

In [91]:
emails_2011_08 = emails.iloc[0:2]
#this creates a small subsection of the emails that were sent in 2011/08 for testingdsfdf

In [21]:
emails.shape

(35942, 28)

### Nodes should be people defined by From, CC, To

In [106]:
nodes_2011_08 = funclib.get_Unique_Nodes(emails_2011_08)

In [59]:
#Found the following emails that need fixing: 
"""
heperez<heperez@earthlink.net%3cmailto:heperez><heperez@earthlink.net%3cmailto:heperez
microsoftexchange329e71ec88ae4615bbc36ab6ce41109e@talgov.com
jay.tallahasseedowntown=gmail.com@mail129.us2.mcsv.net
microsoftexchange329e71ec88ae4615bbc36ab6ce41109e@talgov.com
microsoftexchange329e71ec88ae4615bbc36ab6ce41109e@talgov.com
microsoftexchange329e71ec88ae4615bbc36ab6ce41109e@talgov.com
jay.tallahasseedowntown=gmail.com@mail121.us2.mcsv.net
microsoftexchange329e71ec88ae4615bbc36ab6ce41109e@talgov.com
jay.tallahasseedowntown=gmail.com@mail21.wdc03.rsgsv.net
heperez<heperez@earthlink.net%3cmailto:heperez><heperez@earthlink.net%3cmailto:heperez
jay.tallahasseedowntown=gmail.com@mail66.atl11.rsgsv.net
jay.tallahasseedowntown=gmail.com@mail181.wdc02.mcdlv.net
jay.tallahasseedowntown=gmail.com@mail205.atl81.rsgsv.net
microsoftexchange329e71ec88ae4615bbc36ab6ce41109e@talgov.com
jay.tallahasseedowntown=gmail.com@mail21.wdc03.rsgsv.net
microsoftexchange329e71ec88ae4615bbc36ab6ce41109e@talgov.com
jay.tallahasseedowntown=gmail.com@mail180.wdc02.mcdlv.net
microsoftexchange329e71ec88ae4615bbc36ab6ce41109e@talgov.com
jay.tallahasseedowntown=gmail.com@mail21.wdc03.rsgsv.net
jay.tallahasseedowntown=gmail.com@mail26.wdc03.rsgsv.net
jay.tallahasseedowntown=gmail.com@mail26.wdc03.rsgsv.net
jay.tallahasseedowntown=gmail.com@mail28.wdc03.rsgsv.net
microsoftexchange329e71ec88ae4615bbc36ab6ce41109e@talgov.com
jay.tallahasseedowntown=gmail.com@mail25.wdc03.rsgsv.net
microsoftexchange329e71ec88ae4615bbc36ab6ce41109e@talgov.com
invitation.miami-cslt-owner@liste.diplomatie.gouv.fr
microsoftexchange329e71ec88ae4615bbc36ab6ce41109e@talgov.com
heperez<heperez@earthlink.net%3cmailto:heperez><heperez@earthlink.net%3cmailto:heperez
jay.tallahasseedowntown=gmail.com@mail66.atl11.rsgsv.net
microsoftexchange329e71ec88ae4615bbc36ab6ce41109e@talgov.com
microsoftexchange329e71ec88ae4615bbc36ab6ce41109e@talgov.com
jay.tallahasseedowntown=gmail.com@mail24.wdc03.rsgsv.net
jay.tallahasseedowntown=gmail.com@mail24.wdc03.rsgsv.net
jay.tallahasseedowntown=gmail.com@mail26.wdc03.rsgsv.net
jay.tallahasseedowntown=gmail.com@mail174.us4.mcsv.net
patrickmckee@mindspring.com%3cpatrickmckee@mindspring.com>
850)545-2095edward.kring@talgov.comsentfrommyiphonethanks,eddieedwardmarwkringci-zenengagementambassador,departmentofcommunica-onsoﬃce:(850)891-8081cell:(850
850)545-2095edward.kring@talgov.comsentfrommyiphonethanks,eddieedwardmarwkringci-zenengagementambassador,departmentofcommunica-onsoﬃce:(850)891-8081cell:(850
downtown_colleagues@mailman.downtowndevelopment.com
downtown_colleagues@mailman.downtowndevelopment.com
downtown_colleagues@mailman.downtowndevelopment.com
""";

In [132]:
all_nodes = funclib.get_Unique_Nodes(emails, verbose=False)
#generates the nodes that were used in 2011/08 and all of the nodes in the dataset
len(all_nodes)

25074

In [ ]:
dfg.size().index

In [158]:
# Only keep From/To/CC with emails
def make_edges_ge(df):
    dfg = emails.groupby('From')
    edges = defaultdict(list)

    for from_node in dfg.size().index:
        if not rex.match(r'.*@', from_node): 
            continue
        db = dfg.get_group(from_node)
        edges[from_node] = list(funclib.get_receiver_nodes(db, from_node))
    return edges

In [161]:
edges1 = make_edges_ge(emails)
print(len(edges1))
list(edges1.keys())

enter 
1836


collections.defaultdict

In [ ]:
edge_2011_08 = make_edgelist(emails_2011_08, nodes_2011_08)
#generates an edgelist based on the nodes generated on the emails_2011_08 df

In [61]:
funclib.make_edgelist(emails,all_nodes);

In [ ]:

years = sep_by_year(emails)

In [ ]:
month_2012 = sep_by_month(years[1])

In [ ]:
sep_by_day(month_2012[0])

In [ ]:
disp_Graph(nodes_2011_08, edge_2011_08, '2011/08')
display(edge_2011_08)
#displays the graph based on the first month of the dataset

In [ ]:
nodes_2012_01 = get_Unique_Nodes(month_2012[0])
edges_2012_01 = make_edgelist(month_2012[0], nodes_2012_01)


disp_Graph(nodes_2012_01,edges_2012_01, '2012/01')

#this displays the emails in the second month of the dataset
#though as can be seen here the graph gets very messy when too many nodes are added

In [ ]:
display(len(nodes_2012_01))

In [ ]:
#this is to test out different ways of concating dfs
all_2012 = pd.DataFrame()
for i in range(12):
  all_2012 = all_2012.append(month_2012[i])
display(len(all_2012))

In [ ]:
nodes_2012 = get_Unique_Nodes(all_2012)
edges_2012 = make_edgelist(all_2012, nodes_2012)

In [ ]:
x = edges_2012[0]
display(len(x))

In [ ]:
create_edges_txt(edges_2012)

In [ ]:
create_nodes_txt(nodes_2012)

In [ ]:
all_2012_graph = Graph()
all_2012_graph.read_edges('edges.txt')
all_2012_graph.read_node_metadata('nodes.txt')

In [ ]:
model = Simple_GNN(16,64,8133,3,aggregation_mode='mean')

display((all_2012_graph.node_metadata).shape)

### We initialize the GNN_Trainer and train for 200
### epochs on CPU. 
trainer = GNN_Node_Trainer(all_2012_graph,num_epochs=200,random_seed=1)
#display(model)
predictions = trainer(model)

In [ ]:
display(edge_2011_08)
get_sent_list(edge_2011_08, len(nodes_2011_08))
sent_2012 = get_sent_list(edges_2012, len(nodes_2012))

In [ ]:
recieved_2012 = get_recieved_list(edges_2012, len(nodes_2012))

In [ ]:
node_num_2012 = []
for node in nodes_2012:
  x = nodes_2012.index(node)
  node_num_2012.append(x)
node_meta_2012 = [node_num_2012, sent_2012, recieved_2012]
display(len(node_meta_2012))
create_nodes_txt(node_meta_2012)

In [ ]:
all_2012_graph = Graph()
all_2012_graph.read_edges('edges.txt')
all_2012_graph.read_node_metadata('nodes.txt')

In [ ]:
model = Simple_GNN(2,64,5958,3,aggregation_mode='mean')

display((all_2012_graph.node_metadata).shape)

### We initialize the GNN_Trainer and train for 200
### epochs on CPU. 
trainer = GNN_Node_Trainer(all_2012_graph,num_epochs=200,random_seed=1)
#display(model)
predictions = trainer(model)

In [ ]:
email_node_meta_2012 = get_email_node_meta(all_2012)

In [ ]:
email_edge_2012 = create_email_edgelist(all_2012, nodes_2012)

In [ ]:
email_G = nx.Graph()
email_nodes_2012 = []
for i in range(len(all_2012)):
  email_nodes_2012.append(i)
email_G.add_nodes_from(email_nodes_2012)
email_G.add_edges_from(email_edge_2012)
for i in range(len(all_2012)):
  email_G.nodes[i]['num char'] = email_node_meta_2012[0][i]
  email_G.nodes[i]['num word'] = email_node_meta_2012[1][i]
  email_G.nodes[i]['mean char in word'] = email_node_meta_2012[2][i]
  email_G.nodes[i]['std char in word'] = email_node_meta_2012[3][i]

email_G.nodes.data()

In [ ]:
test_node_11_08 = create_node_list(emails_2011_08)
display(test_node_11_08)

In [ ]:
test_edge_11_08 = create_edge_list(emails_2011_08, test_node_11_08)
display(test_edge_11_08)

In [ ]:
test_node_2012_01 = create_node_list(month_2012[0])
test_edge_2012_01 = create_edge_list(month_2012[0], test_node_2012_01)

display(test_node_2012_01)
display(test_edge_2012_01)

In [ ]:
x = []
display(len(x))

In [ ]:
rec_node_list = get_node_recieved_list(month_2012[0], test_node_2012_01)
display(rec_node_list)

In [ ]:
test_node_meta_2012_01 = create_node_meta_list(month_2012[0], test_node_2012_01)
display(test_node_meta_2012_01)

In [ ]:
create_edge_meta_list(month_2012[0], test_node_2012_01, test_edge_2012_01)

In [ ]:
node_list_2012 = create_node_list(all_2012)
edge_list_2012 = create_edge_list(all_2012, node_list_2012)
node_meta_list_2012 = create_node_meta_list(all_2012, node_list_2012)
rec_node_list_2012 = get_node_recieved_list(all_2012, node_list_2012)

In [ ]:
bi_class_to_2012 = create_bi_class_to(all_2012, node_meta_list_2012)

In [ ]:
#later create a more optimized way of seperating the node metadata
char_mean_2012 = []
word_mean_2012 = []
to_mean_2012 = []
cc_mean_2012 = []
char_std_2012 = []
word_std_2012 = []
to_std_2012 = []
cc_std_2012 = []
num_sent_2012 = []
#rec_node_list_2012
bi_class_char_2012 = create_bi_class_char(all_2012, node_meta_list_2012)
for i in range(len(node_meta_list_2012)):
  q = node_meta_list_2012[i]
  char_mean_2012.append(q[0])
  word_mean_2012.append(q[1])
  to_mean_2012.append(q[2])
  cc_mean_2012.append(q[3])
  char_std_2012.append(q[4])
  word_std_2012.append(q[5])
  to_std_2012.append(q[6])
  cc_std_2012.append(q[7])
  num_sent_2012.append(q[8])
bi_class_list_char_mean_2012 = create_bi_class(all_2012, char_mean_2012)
bi_class_list_word_mean_2012 = create_bi_class(all_2012, word_mean_2012)
bi_class_list_to_mean_2012 = create_bi_class(all_2012, to_mean_2012)
bi_class_list_cc_mean_2012 = create_bi_class(all_2012, cc_mean_2012)
bi_class_list_char_std_2012 = create_bi_class(all_2012, char_std_2012)
bi_class_list_word_std_2012 = create_bi_class(all_2012, word_std_2012)
bi_class_list_to_std_2012 = create_bi_class(all_2012, to_mean_2012)
bi_class_list_cc_std_2012 = create_bi_class(all_2012, cc_mean_2012)
bi_class_list_num_sent_2012 = create_bi_class(all_2012, num_sent_2012)
bi_class_list_num_rec_2012 = create_bi_class(all_2012, rec_node_list_2012)
bi_classes = [bi_class_list_char_mean_2012, bi_class_list_word_mean_2012, bi_class_list_to_mean_2012, bi_class_list_cc_mean_2012, bi_class_list_char_std_2012, bi_class_list_word_std_2012, bi_class_list_to_std_2012, bi_class_list_cc_std_2012, bi_class_list_num_sent_2012, bi_class_list_num_rec_2012]
display(len(node_meta_list_2012))
display(len(node_meta_list_2012[0]))
display(node_meta_list_2012[4])

In [ ]:
#this is what generated the plots that I used in the meeting


titles = ['mean char', 'mean words', 'mean to', 'mean cc', 'std char', 'std words', 'std to', 'std cc', 'num sent', 'num recieved']
figure_title = ['mean_char.png', 'mean_words.png', 'mean_to.png', 'mean_cc.png', 'std_char.png', 'std_words.png', 'std_to.png', 'std_cc.png', 'num_sent.png', 'num_recieved.png']
for i in range(10):
  create_node_list_txt(node_list_2012, node_meta_list_2012, rec_node_list_2012, bi_classes[i])
  create_edges_txt(edge_list_2012)
  all_2012_graph = Graph()
  all_2012_graph.read_edges('edges.txt')
  all_2012_graph.read_node_metadata('nodes.txt')
  model = Simple_GNN(10,64,2,3,aggregation_mode='mean')

  display((all_2012_graph.node_metadata).shape)
  

  ### We initialize the GNN_Trainer and train for 200
  ### epochs on CPU. 
  r = torch.randperm(all_2012_graph.num_nodes)
  trainer = GNN_Node_Trainer(all_2012_graph,train_index= r[:int(.8 * len(r))], test_index= r[int(.8 * len(r))::],num_epochs=200,random_seed=1)
  #display(model)
  predictions = trainer(model)
  print(f"Train Class #: {all_2012_graph.node_metadata[:,0][trainer.train_index].unique(return_counts=True)[1]}")
  print(f"Test Class #: {all_2012_graph.node_metadata[:,0][trainer.test_index].unique(return_counts=True)[1]}")
  acc = print_acc(predictions, bi_classes[i])
  color_str = 'rb'
  #colors = cm.rainbow(predictions.argmax(dim=1))
  predx = []
  predy = []
  prepx1 = []
  prepx2 = []
  prepy1 = []
  prepy2 = []
  for j in range(len(predictions)):
    q = torch.detach(predictions[j])
    predx.append(q[0])
    predy.append(q[1])
    if predictions.argmax(dim=1)[j] == 0:
      prepx1.append(q[0])
      prepy1.append(q[1])
    else:
      prepx2.append(q[0])
      prepy2.append(q[1])
  plt.figure()
  e = np.ones(len(predictions))#0 is purple 1 is yellow
  #plt.scatter(predx, predy, c=e)
  #plt.scatter(predx, predy, c=predictions.argmax(dim=1))
  fig, ax = plt.subplots()
  p1 = ax.scatter(prepx1, prepy1, c='blue')
  p2 = ax.scatter(prepx2, prepy2, c='red')
  s = str(acc)
  plt.title(titles[i] + ' accuraccy = ' + s[0:5] + '%')
  fig.legend((p1,p2),('less than','greater than'))
  fig.savefig(figure_title[i])
  plt.show()
  


In [ ]:
create_node_list_txt(node_list_2012, node_meta_list_2012, rec_node_list_2012, bi_class_to_2012)
create_edges_txt(edge_list_2012)


In [ ]:
all_2012_graph = Graph()
all_2012_graph.read_edges('edges.txt')
all_2012_graph.read_node_metadata('nodes.txt')
model = Simple_GNN(10,64,2,3,aggregation_mode='mean')

display((all_2012_graph.node_metadata).shape)

### We initialize the GNN_Trainer and train for 200
### epochs on CPU. 
trainer = GNN_Node_Trainer(all_2012_graph,num_epochs=200,random_seed=1)
#display(model)
predictions = trainer(model)

In [ ]:
display(torch.detach(predictions[0]))

In [ ]:
create_node_list_txt(node_list_2012, node_meta_list_2012, rec_node_list_2012, bi_class_char_2012)
create_edges_txt(edge_list_2012)

In [ ]:
all_2012_graph = Graph()
all_2012_graph.read_edges('edges.txt')
all_2012_graph.read_node_metadata('nodes.txt')
model = Simple_GNN(10,64,2,3,aggregation_mode='mean')

display((all_2012_graph.node_metadata).shape)

### We initialize the GNN_Trainer and train for 200
### epochs on CPU. 
trainer = GNN_Node_Trainer(all_2012_graph,num_epochs=200,random_seed=1)
#display(model)
predictions = trainer(model)

In [ ]:
print(f"Train Class #: {all_2012_graph.node_metadata[:,0][trainer.train_index].unique(return_counts=True)[1]}")
print(f"Test Class #: {all_2012_graph.node_metadata[:,0][trainer.test_index].unique(return_counts=True)[1]}")

In [ ]:
email_G_2012 = nx.Graph()
email_nodes_list_2012 = []
for i in range(len(node_list_2012)):
  email_nodes_list_2012.append(i)
email_G_2012.add_nodes_from(email_nodes_list_2012)
email_G_2012.add_edges_from(edge_list_2012)
for i in range(len(all_2012)):
  email_G_2012.nodes[i]['mn num char'] = node_meta_list_2012[i][0]
  email_G_2012.nodes[i]['mn num word'] = node_meta_list_2012[i][1]
  email_G_2012.nodes[i]['mn to'] = node_meta_list_2012[i][2]
  email_G_2012.nodes[i]['mn cc'] = node_meta_list_2012[i][3]
  email_G_2012.nodes[i]['std num char'] = node_meta_list_2012[i][4]
  email_G_2012.nodes[i]['std num word'] = node_meta_list_2012[i][5]
  email_G_2012.nodes[i]['std to'] = node_meta_list_2012[i][6]
  email_G_2012.nodes[i]['std cc'] = node_meta_list_2012[i][7]
  email_G_2012.nodes[i]['sent num'] = node_meta_list_2012[i][8]
  email_G_2012.nodes[i]['recieved num'] = rec_node_list_2012[i]
email_G_2012.nodes.data()

In [ ]:
# 0: training set (50%); 1: testing set (50%)
# mask = torch.from_numpy(np.random.randint(0, 2, H0.shape[0]))

def setup_training(model, mask, frac_train=0.5, lr=0.001):
    mask = torch.from_numpy(
        np.random.choice(2, G.nb_nodes, p=[frac_train, 1.0 - frac_train])
    )

    loss_fn = BinaryCrossEntropyLoss(mask)

    # optimizer = torch.optim.SGD(model.parameters(), lr=1.e-3)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    return loss_fn, optimizer

In [ ]:
G = nx.karate_club_graph()
karate.add_gaussian_features(G, means=[0.0, 1.5], std=(1, 1), nb_features=16)
display(G.node_metadata.shape[0])
model = MyGCN(G)
loss_fn, optimizer = setup_training(model, mask, frac_train=0.55, lr=0.001)
losses, accuracy_count = new_train(G, model, mask, loss_fn, optimizer, nb_epochs=1000)